In [1]:
import pickle
import string

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd

import fasttext as ft
import fasttext.util

In [2]:
tf.config.list_physical_devices('GPU'), tf.test.is_built_with_cuda()

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], True)

In [3]:
# Our dataset is around ~15mb
df = pd.read_csv('old_general.csv', header=None, usecols=[2], names=['chat'])
df.head(10)

,chat
0,Nigger in the house
1,Nigger
2,"""Yes"""
3,I have a hard on right now
4,same tbh
5,im actually jerking off rn
6,I'm fapping to cosplaying kids
7,same
8,con season is best season
9,true


In [4]:
df['chat'] = df['chat'].apply(lambda x: x.lower())
df['chat'] = df['chat'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

BOS = 'xxbos'
EOS = 'xxeos'
UNK = 'xxunk'

def add_utility_tokens(r):
    content = r['chat']

    return f'{BOS} {content} {EOS}'

df['chat'] = df.map(add_utility_tokens, axis=1)

df.head(10)

,chat
0,nigger in the house
1,nigger
2,yes
3,i have a hard on right now
4,same tbh
5,im actually jerking off rn
6,im fapping to cosplaying kids
7,same
8,con season is best season
9,true


In [5]:
df['tkn_len'] = df['chat'].apply(lambda x: len(x.split(' ')))

_p = {i: np.percentile([l for l in df['tkn_len'].values], i) for i in [70, 80, 90, 95, 97, 99]}

_p

{70: 7.0, 80: 9.0, 90: 12.0, 95: 16.0, 97: 19.0, 99: 28.0}

In [14]:
MAX_TOKENS = 20

df = df[df['tkn_len'] <= MAX_TOKENS]

def add_utility_tokens(r):
    content = r['chat']

    return f'{BOS} {content} {EOS}'

df['chat'] = df.map(add_utility_tokens, axis=1)

MAX_TOKENS += 2

df.head(10)

,chat,tkn_len,chat_2
0,xxbos nigger in the house xxeos xxpad xxpad xx...,4,xxbos nigger in the house xxeos xxpad xxpad xx...
1,xxbos nigger xxeos xxpad xxpad xxpad xxpad xxp...,1,xxbos nigger xxeos xxpad xxpad xxpad xxpad xxp...
2,xxbos yes xxeos xxpad xxpad xxpad xxpad xxpad ...,1,xxbos yes xxeos xxpad xxpad xxpad xxpad xxpad ...
3,xxbos i have a hard on right now xxeos xxpad x...,7,xxbos i have a hard on right now xxeos xxpad x...
4,xxbos same tbh xxeos xxpad xxpad xxpad xxpad x...,2,xxbos same tbh xxeos xxpad xxpad xxpad xxpad x...
5,xxbos im actually jerking off rn xxeos xxpad x...,5,xxbos im actually jerking off rn xxeos xxpad x...
6,xxbos im fapping to cosplaying kids xxeos xxpa...,5,xxbos im fapping to cosplaying kids xxeos xxpa...
7,xxbos same xxeos xxpad xxpad xxpad xxpad xxpad...,1,xxbos same xxeos xxpad xxpad xxpad xxpad xxpad...
8,xxbos con season is best season xxeos xxpad xx...,5,xxbos con season is best season xxeos xxpad xx...
9,xxbos true xxeos xxpad xxpad xxpad xxpad xxpad...,1,xxbos true xxeos xxpad xxpad xxpad xxpad xxpad...


In [16]:
# push to tensorflow
df['chat_shift'] = df['chat'].shift(-1)
df.drop(df.tail(1).index, inplace=True) # drop last row for NaN due to shift

dataset = tf.data.Dataset.from_tensor_slices((df['chat'].values, df['chat_shift'].values))

In [18]:
# tokenize and build vocab
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
vocabulary_set.update([BOS, EOS, UNK])

for xb, yb in dataset:
  xb_tokens = tokenizer.tokenize(xb.numpy())
  vocabulary_set.update(xb_tokens)

  yb_tokens = tokenizer.tokenize(yb.numpy())
  vocabulary_set.update(yb_tokens)

vocab_size = len(vocabulary_set)
vocab_size += 1 # for padding introduced dataset.padded_batch (as 0.0f)

vocab_size

43604

In [ ]:
# collate_fn (for the fastai users)
def encode(xb, yb):
  enc_x, enc_y = encoder.encode(xb.numpy()), encoder.encode(yb)
  return enc_x, enc_y

def encode_map_fn(xb, yb):
  enc_xb, enc_yb = tf.py_function(encode, 
                                       inp=[xb, yb], 
                                       Tout=(tf.int64, tf.int64))
  return enc_xb, enc_yb

dataset = dataset.map(encode_map_fn)

In [ ]:
# now you can convert each X-Y pair to
# X-Y[:i]-Y triplets, for i in range(2, len(Y))
# (enc-input, dec-input, dec-output)

# note: if you want to use teacher forcing, you must implement your own 
# training loop: https://keras.io/guides/writing_a_training_loop_from_scratch/

# we just do the first thing for simplicity
def unwrap_for_seq2sec(xb, yb):        
  return np.array([(xb, yb[1:i], yb)  for i in range(1, len(yb))])

dataset = dataset.map(map_func=lambda xb, yb: tf.py_func(
  func=unwrap_for_seq2sec, inp=[xb, yb], Tout=(tf.int64, tf.int64, tf.int64)
))

In [ ]:
TAKE_SIZE = 20 # just to visualize some in the end
BUFFER_SIZE = 1024
BATCH_SIZE = 256
EPOCHS = 15

train_data = encoded_dataset.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = encoded_dataset.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [ ]:
# load fasttext model and build embeddings layer
# ft.util.download_model('en', if_exists='ignore')
en_vecs = ft.load_model('./cc.en.300.bin')
en_vecs.get_dimension() # 300

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))

for word in vocabulary_set:
    i = encoder.encode(word)
    embedding_matrix[i] = en_vecs.get_word_vector(word)

embedding_layer = tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix])

In [ ]:
# https://keras.io/examples/lstm_seq2seq/

latent_dim = 256

# input is (tokens, vocab size)
encoder_input = Input(shape=(None, MAX_TOKENS, vocab_size))

# embedding dim is 300 from the fasttext model
encoder_embedding = embedding_layer(encoder_input)

encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embedding)

encoder_states = [state_h, state_c]

# same logic as model_input
decoder_input = Input(shape=(None, MAX_TOKENS, vocab_size))

decoder_embedding = decoder_input

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding,
                                     initial_state=encoder_states)

decoder_dense = Dense(vocab_size * MAX_TOKENS, activation='softmax')
decoder_output = decoder_dense(decoder_outputs)

model = Model([encoder_input, decoder_input], decoder_output)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# build dataset records

In [19]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS)

NameError: name 'model' is not defined

In [20]:
# visualize some results from test data

# for inference just:
# 1. ensure input has got maximum MAX_TOKENS length
# 2. create a tensor of length MAX_TOKENS
# 3. paste the encoding tokens on the beginning of the tensor
# 4. until the EOS token is encountered on the output or when length = MAX_TOKENS
# 5. get the results of the output and plug it in the input to get next decoder input

In [22]:
model.save('for_inference', save_format='tf')
encoder.save_to_file('text.encoder')